In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, pipeline
import torch

# Specify the directory where your fine-tuned model and tokenizer are saved
model_dir = "./gen_model/checkpoint-230"

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_dir,)

# Load the model
model = GPT2LMHeadModel.from_pretrained(model_dir)

# Optionally, you can move the model to the appropriate device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

/Users/frederiklamp/OneDrive - Syddansk Universitet/Kandidat_Data_Science_SDU/Specialemappe/Nic_Chr_group/project/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-31 12:15:52.259473: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
import pickle

# Load the test data
test_data = pickle.load(open("./data/test_data_10p.pkl", "rb")) # load from pickle file to avoid reprocessing

print(test_data[:10])

Function to reprocess the test data strings into 3 sections: user, review, reply

In [8]:
def separate_sections(text):
    # Define the tags
    tags = ["### Bruger:", "### Anmeldelse:", "### Svar:"]
    
    # Initialize an empty list to store the sections
    sections = ["", "", ""]
    
    # Split the text by the first tag to get the user's name section
    parts = text.split(tags[0])
    if len(parts) > 1:
        sections[0] = parts[1].split(tags[1])[0].strip()
    
    # Split the text by the second tag to get the review section
    parts = text.split(tags[1])
    if len(parts) > 1:
        sections[1] = parts[1].split(tags[2])[0].strip()
    
    # Split the text by the third tag to get the response section
    parts = text.split(tags[2])
    if len(parts) > 1:
        sections[2] = parts[1].strip()
    
    return sections

In [ ]:
example_reviews = [separate_sections(review) for review in test_data[:10]]
for i in example_reviews:
    print(i)

Define function for stripping excess generated text

In [14]:
###For testing
def prompt_template(user, review):
    return f"### Bruger:\n{user}\n\n### Anmeldelse:\n{review}\n\n### Svar:\nKære {user}\n"

import re

# Generate text but only return one reply and strip the rest
def extract_first_lines(generated_text):
    # Defining regex patterns for each of the three tags
    patterns = {
        '### Bruger:': r'### Bruger:\s*(.*)',
        '### Anmeldelse:': r'### Anmeldelse:\s*(.*)'
    }
    
    # Extract the first occurrence of '### Bruger:' and '### Anmeldelse:'
    extracted_content = {}
    for tag, pattern in patterns.items():
        match = re.search(pattern, generated_text, re.MULTILINE)
        if match:
            extracted_content[tag] = match.group(1).strip()
    
    # Extract the entire text between '### Svar:' and the next '### Bruger:' or '### Anmeldelse:'
    response_pattern = r'### Svar:\s*(.*?)((?=### Bruger:|### Anmeldelse:|### svar)|$)'
    response_match = re.search(response_pattern, generated_text, re.DOTALL)
    if response_match:
        extracted_content['### Svar:'] = response_match.group(1).strip()
    
    return extracted_content

Now get the predictions (generated replies) from the model on the first part of the test data:

In [22]:
predictions = []
for review in example_reviews:
    # Generate text
    user = review[0]
    review_sequence = review[1]

    # Using pipeline
    generator = pipeline("text-generation", model= model, tokenizer=tokenizer)

    prompt = prompt_template(user, review_sequence)

    # get the output
    output = generator(prompt, max_length= 500, do_sample=True, temperature=0.4) # low temp for less randomness
    
    # select output text
    generated_text = (output[0]['generated_text'])

    # make sure to remove excess text (only one of each '###' tag per review)
    extracted_content = extract_first_lines(generated_text)

    # get the text from the response tag: '###svar: ' (actual reply)
    response = extracted_content['### Svar:']

    #append the response to the predictions list
    predictions.append('### Svar :' + response)
    


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
for i in predictions[:5]:
    print(i)

Feel free to experiment the performance of the model by generating outputs to different types of reviews.

The review should be related to a customer experience with buying a car or from  a mechanic service appointment.

In [7]:
import pandas as pd
# Test sequence
one_star_reviews = {
    'bruger': ['Birthe', 'Michael', 'Mette'], 
    'review': ["Holder ikke hvad de lover. Bryder aftaler og giver kun en problemer og gæld med hjem. Kan ikke stole på dem.",
    "Jeg sender min bil til service og syn hvor jeg så for et opkald om at mine stabilisator arme skal skiftes grundet slør samt skover og bremser bagpå skal skiftes, det forlanger de omkring 7000 for. Hvortil jeg afviser at de skal lave den. Også af den grund at jeg ikke har haft noget slør i bilen inden den kom til service. \nDa jeg så får bilen tilbage er der pludselig lyde fra mine stabilisator arme samt at der pludselig er kommet slør i rettet.",
    "Vi leverede en bil til service og skade reparation. Da vi hentede kunne bremsen trædes helt ned. Retur igen. Da vi hentede anden gang lyste en masse lamper, så retur igen. \n\nVi er utilfredse med at alt ikke bliver lavet i første omgang, når vi afleverer en bil."
    ],
}
two_star_reviews = {
    'bruger': ['Louise', 'Bertram', 'Simon'],
    'review': ["Har købt en suzuki baleno for 2 år siden han tog en toyota aygo i bytte kan regne ud at det var alt for lidt vi fik for den og baleno var alt for dyr i forhold til hvad den er værd idag selvom vi lagde en stor udbetaling.\nVi føler os snydt, derfor bliver det ikke et sted vi handler igen",
               "Venlig betjening men skuffende resultat. 6 år gammel bil bliver kasseret til syn mindre end 3 uger efter på en mekanisk fejl.",
               "En bule i skærmen skal repareres, sammen med et service. efter 2 ugers ventetid og 4 forsøg på at ringe værkstedet op, får jeg at vide, at der ikke er nogen tidshorisont for min bil. De har fået den forkerte reservedel og aner ikke hvornår de får den rigtige."
    ],
}
three_star_reviews = {
    'bruger': ['Connie', 'Mads', 'Gitte'],
    'review': ["Ok service. God pris når man skal have fundet fejlen på bilen",
               "Service og rådgivning fint. Desværre var efterfølgende prisoverslag markant dyrere end tilsvarende værksteder så de ekstra ting som skulle laves måtte jeg løse andetsteds.",
               "Min reparation blev diagnosticeret og reservedele ( nogle slanger) hentet hjem, men af 2 omgange var det de forkerte, så jeg måtte vente 2 dage ekstra, herefter blev en ny fejl, der ville tredoble prisen fundet. \nEfterfølgende blev slangerne fejlmonteret så diesel fossede ud i motor og under bilen. Eneste grund til de ikke får 1 stjerne for dette arbejde er deres meget fine måde at være på under hele forløbet, beklagende og forstående. God service, meget tvivlsomt arbejde."
    ],
}

four_star_reviews = {
    'bruger': ['Mikkel', 'Jens', 'Anders'],
    'review': ["Forsikringsreparation. Den bestilte forsikringsreparation ganske ok, fin bilvask - MEN lånebilen lod en del tilbage at ønske",
               "Hurtig, god og serviceminded kontakt og booking.\nReparationen er veludført og uden efterfølgende problemer. \nOplevelsen med lånebilen var dog ikke fantastisk, der var 34 km tilbage i tanken da jeg fik den og koblingen virkede kun på den sidste cm af pedalgangen.",
               "Ok oplevelse. \nGod information hele processen igennem! \nDe fik min gamle bil til at køre igen, jeg er glad"
    ]
}
five_star_reviews = {
    'bruger': ['Casper', 'Sofie', 'Lars'],
    'review': ["Tager sig tid til at give grundig information om reparationer med videoer forklaringer mv. Meget venligt og serviceminded personale. Hurtig service. Jeg føler mig generelt meget tryg ved værkstedet.",
               "Super service og en butik hvor man føler sig velkommen.",
               "Anbefales!!\nMin sælger var enormt forstående, det var første bil jeg skulle eje så der var lidt forskelligt men han tog vores bekymringer til sig. Hurtig svar og info vedrørende information om både levering og syn."
    ]
}
review_list = [one_star_reviews, two_star_reviews, three_star_reviews, four_star_reviews, five_star_reviews]

In [ ]:
responses = []

for dict in review_list:
    for user, review in zip(dict['bruger'], dict['review']):

        test_sequence_review = review

        #convert prompt with template
        test_prompt = prompt_template(user = user, review = test_sequence_review)

        # Using pipeline
        generator = pipeline("text-generation", model= model, tokenizer=tokenizer)

        # Generate text
        test_reply_list = generator(test_prompt, max_length= 300, do_sample=True, temperature=0.4)

        # Extract the generated reply
        test_reply = (test_reply_list[0]['generated_text'])

        # Strip the generated reply to exclude excess text
        extracted_content = extract_first_lines(test_reply)

        reply_object= []
        reply = ""
        # Print the extracted content
        for tag, content in extracted_content.items():
            reply += f"{tag} {content}"
        reply_object.append(reply)
        print(reply_object)
        #replies.append(reply_object)

        response = extracted_content['### Svar:']
        responses.append('### Svar :' + response)

        break

print(responses)

#df = pd.DataFrame(replies, columns=['User', 'Review', 'Reply'])
#df.to_csv('generated_replies.csv', index=False, sep=';')